<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/ELYZA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 準備
長いテキストを折り返し表示させる命令

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# 大規模言語モデルの実習
chatGPT が禁止なので、Llama 2 を使います。

## Llama 2とは
Llama 2とは、2023年7月18日にMeta社が公開した英語ベースの大規模言語モデルです。先に公開された「LLaMA」が研究用途に限定されていたのに対し、Llama 2は商用利用も可能となっています。公開されているモデルとしてはとても性能が高いことから、OpenAIのGPT-4やGoogleのPaLMなどのクローズドなLLMと競合する形で、英語圏では既にオープンモデルのデファクトスタンダードとなりつつあります。

サイズは70億、130億、700億の3種類となっており、いずれのモデルも教師ありファインチューニング（Supervised Fine-Tuning、SFT）及び、人間からのフィードバックに基づいた強化学習（Reinforcement Learning from Human Feedback、RLHF）を施したchatモデルも同時に公開しています。


## ELYZA-japanese-Llama-2-7b とは

「ELYZA-japanese-Llama-2-7b」は、東京大学松尾研究室発・AIスタートアップの「ELYZA」が開発した、日本語LLMです。Metaの「Llama 2」をベースに、日本語による追加事前学習を行なった日本語言語モデル「ELYZA-japanese-Llama-2-7b」と、そこにELYZA独自の事後学習を施した「ELYZA-japanese-Llama-2-7b-instruct」、日本語の語彙追加により高速化を行った「ELYZA-japanese-Llama-2-7b-fast / ELYZA-japanese-Llama-2-7b-fast-instruct」を一般公開しました。いずれも70億パラメータのモデルで、公開されている日本語のLLMとしては最大級の規模です。

ライセンスはLlama 2 Community License に準拠しており、Acceptable Use Policy に従う限りにおいては、研究および商業目的での利用が可能です。

### パッケージのインストール

In [ ]:
!pip install transformers accelerate bitsandbytes

### トークナイザーとモデルの準備。
今回は、高速な指示モデル (elyza/ELYZA-japanese-Llama-2-7b-fast-instruct) を利用しています。

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained(
    "elyza/ELYZA-japanese-Llama-2-7b-instruct"
)
model = AutoModelForCausalLM.from_pretrained(
    "elyza/ELYZA-japanese-Llama-2-7b-instruct",
    torch_dtype=torch.float16,
    device_map="auto"
)

### プロンプトの書式

```
<s>[INST] <<SYS>>
{システムメッセージ}
<</SYS>>
{ユーザーメッセージ}[/INST]
```

### テキスト生成

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。"

# ストリーミング出力
def ask(text):
    prompt = "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
        bos_token=tokenizer.bos_token,
        b_inst=B_INST,
        system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
        prompt=text,
        e_inst=E_INST,
    )

    with torch.no_grad():
        streamer = TextIteratorStreamer(tokenizer)
        token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
        generation_kwargs = dict(
            input_ids=token_ids.to(model.device),
            max_new_tokens=1024,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            streamer=streamer,
        )
        thread = Thread(target=model.generate, kwargs=generation_kwargs)
        thread.start()

    for new_text in streamer:
        print(new_text.replace("[/INST]", "[/INST]\n\n"), end="")

In [ ]:
model.generation_config

### 質問と答えの例

In [ ]:
ask("横須賀市の観光プランを考えて下さい。")

In [ ]:
ask("カレーの作り方を教えて下さい。")

### 英語で聞くと英語で返してくる

In [ ]:
ask("Please answer the following quiz question. The combined price of a ballpoint pen and an eraser is 110 yen. The ballpoint pen is 100 yen more expensive than the eraser. So, what is the price of the eraser?")

### 複数行の文字列は """ を使う

In [ ]:
ask("""以下の選択肢の中で、日本が発祥とされ、世界中で知られている食べ物は何でしょうか？

A. ピザ
B. パスタ
C. ハンバーガー
D. 寿司
""")

### 小説も書く

In [ ]:
ask("クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。")

# 課題
いろいろ試してみて下さい。